---
title: "Kaggle_exercise_03"
author: "minstar"
date: "2023-05-17"
categories: [python]
---
kaggle 문제풀이 03
https://www.kaggle.com/code/lsiina/exercise-interactive-maps/edit

# 소개

당신은 일본의 도시 안전 기획자이며, 일본의 어떤 지역이 지진 강화를 위해 추가적인 조치가 필요한지 분석하고 있습니다. 인구 밀도가 높으면서 지진이 발생하기 쉬운 지역은 어디인가요?

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/Kuh9gPj.png" width="450"><br/>
</center>
시작하기 전에 아래의 코드 셀을 실행하여 모든 설정을 완료하세요.

In [ ]:
!pip install geopandas

In [ ]:
import pandas as pd
import geopandas as gpd

import folium
from folium import Choropleth
from folium.plugins import HeatMap

인터랙티브 지도를 표시하는 `embed_map()` 함수를 정의합니다. 이 함수는 두 개의 인수를 받습니다: 지도를 담고 있는 변수와 지도가 저장될 HTML 파일의 이름입니다.

이 함수는 지도가 [모든 웹 브라우저에서](https://github.com/python-visualization/folium/issues/812) 표시될 수 있도록 합니다.

In [ ]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='1000px')

# 연습문제

### 1) 지진은 플레이트 경계와 일치하는가?

아래의 코드 셀을 실행하여 전 세계 플레이트 경계를 보여주는 DataFrame `plate_boundaries`를 생성하세요. "coordinates" 열은 경계에 따라 (위도, 경도) 위치의 리스트입니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
plate_boundaries = gpd.read_file("/content/drive/MyDrive/2023 데이터마이닝/dataset/Plate_Boundaries/Plate_Boundaries.shp")
plate_boundaries['coordinates'] = plate_boundaries.apply(lambda x: [(b,a) for (a,b) in list(x.geometry.coords)], axis='columns')
plate_boundaries.drop('geometry', axis=1, inplace=True)

plate_boundaries.head()

,HAZ_PLATES,HAZ_PLAT_1,HAZ_PLAT_2,Shape_Leng,coordinates
0,TRENCH,SERAM TROUGH (ACTIVE),6722,5.843467,"[(-5.444200361999947, 133.6808931800001), (-5...."
1,TRENCH,WETAR THRUST,6722,1.829013,"[(-7.760600482999962, 125.47879802900002), (-7..."
2,TRENCH,TRENCH WEST OF LUZON (MANILA TRENCH) NORTHERN ...,6621,6.743604,"[(19.817899819000047, 120.09999798800004), (19..."
3,TRENCH,BONIN TRENCH,9821,8.329381,"[(26.175899215000072, 143.20620700100005), (26..."
4,TRENCH,NEW GUINEA TRENCH,8001,11.998145,"[(0.41880004000006466, 132.8273013480001), (0...."


다음으로, 아래의 코드 셀을 수정하지 말고 실행하여 역사적인 지진 데이터를 DataFrame `earthquakes`에 로드하세요.

In [ ]:
# Load the data and print the first 5 rows
earthquakes = pd.read_csv("/content/drive/MyDrive/2023 데이터마이닝/dataset/earthquakes1970-2014.csv", parse_dates=["DateTime"])
earthquakes.head()

,DateTime,Latitude,Longitude,Depth,Magnitude,MagType,NbStations,Gap,Distance,RMS,Source,EventID
0,1970-01-04 17:00:40.200,24.139,102.503,31.0,7.5,Ms,90.0,NaN,NaN,0.0,NEI,1.970010e+09
1,1970-01-06 05:35:51.800,-9.628,151.458,8.0,6.2,Ms,85.0,NaN,NaN,0.0,NEI,1.970011e+09
2,1970-01-08 17:12:39.100,-34.741,178.568,179.0,6.1,Mb,59.0,NaN,NaN,0.0,NEI,1.970011e+09
3,1970-01-10 12:07:08.600,6.825,126.737,73.0,6.1,Mb,91.0,NaN,NaN,0.0,NEI,1.970011e+09
4,1970-01-16 08:05:39.000,60.280,-152.660,85.0,6.0,ML,0.0,NaN,NaN,NaN,AK,NaN


아래의 코드 셀은 지도 상에 플레이트 경계를 시각화합니다. 플레이트 경계와 지진이 일치하는지 확인하기 위해 모든 지진 데이터를 사용하여 같은 지도에 히트맵을 추가하세요.

In [ ]:
# Create a base map with plate boundaries
m_1 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)
for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_1)

# Your code here: Add a heatmap to the map
HeatMap(data=earthquakes[['Latitude', 'Longitude']], radius=15).add_to(m_1)

# Uncomment to see a hint
#q_1.a.hint()

# Show the map
embed_map(m_1, 'q_1.html')

위의 지도를 기반으로 지진이 플레이트 경계와 일치하는지 확인해보면 어떻게 되나요?

### 2) 일본에서 지진 깊이와 판 경계에 근접하는 것 사이에 관계가 있습니까?


일본에서 지진의 깊이와 판 경계와의 근접성 사이에 관련이 있는지 궁금합니다. 최근에 읽은 바에 따르면, 지진의 깊이는 지구의 구조에 대한 [중요한 정보](https://www.usgs.gov/faqs/what-depth-do-earthquakes-occur-what-significance-depth?qt-news_science_products=0#qt-news_science_products)를 제공한다고 합니다. 전 세계적으로 흥미로운 패턴이 있는지 확인하고, 또한 일본에서 깊이가 어떻게 변하는지 이해하고 싶습니다.





In [ ]:
# Create a base map with plate boundaries
m_2 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)
for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_2)
    
# Your code here: Add a map to visualize earthquake depth
# Custom function to assign a color to each circle
def color_producer(val):
    if val < 50:
        return 'forestgreen'
    elif val < 100:
        return 'darkorange'
    else:
        return 'darkred'

# Add a map to visualize earthquake depth
for i in range(0,len(earthquakes)):
    folium.Circle(
        location=[earthquakes.iloc[i]['Latitude'], earthquakes.iloc[i]['Longitude']],
        radius=2000,
        color=color_producer(earthquakes.iloc[i]['Depth'])).add_to(m_2)

# Uncomment to see a hint
#q_2.a.hint()

# View the map
embed_map(m_2, 'q_2.html')

판 경계와 지진의 깊이 사이에 관계를 발견할 수 있나요? 이러한 패턴은 전 세계적으로 유지되나요? 일본에서도 마찬가지인가요?

### 3) 인구 밀도가 높은 현은 어디입니까?

다음 코드 셀을 실행하여 (변경하지 않고) 일본 지방의 지리적 경계를 포함하는 GeoDataFrame `prefectures`를 생성하세요.

In [ ]:
# GeoDataFrame with prefecture boundaries
prefectures = gpd.read_file("/content/drive/MyDrive/2023 데이터마이닝/dataset/japan-prefecture-boundaries/japan-prefecture-boundaries.shp")
prefectures.set_index('prefecture', inplace=True)
prefectures.head()

,geometry
prefecture,
Aichi,"MULTIPOLYGON (((137.09523 34.65330, 137.09546 ..."
Akita,"MULTIPOLYGON (((139.55725 39.20330, 139.55765 ..."
Aomori,"MULTIPOLYGON (((141.39860 40.92472, 141.39806 ..."
Chiba,"MULTIPOLYGON (((139.82488 34.98967, 139.82434 ..."
Ehime,"MULTIPOLYGON (((132.55859 32.91224, 132.55904 ..."


다음 코드 셀은 일본 각 지방의 인구, 면적 (제곱 킬로미터), 인구 밀도 (제곱 킬로미터당)를 포함하는 DataFrame `stats`를 생성합니다. 코드 셀을 수정하지 않고 실행하세요.

In [ ]:
# DataFrame containing population of each prefecture
population = pd.read_csv("/content/drive/MyDrive/2023 데이터마이닝/dataset/japan-prefecture-population.csv")
population.set_index('prefecture', inplace=True)

# Calculate area (in square kilometers) of each prefecture
area_sqkm = pd.Series(prefectures.geometry.to_crs(epsg=32654).area / 10**6, name='area_sqkm')
stats = population.join(area_sqkm)

# Add density (per square kilometer) of each prefecture
stats['density'] = stats["population"] / stats["area_sqkm"]
stats.head()

,population,area_sqkm,density
prefecture,,,
Tokyo,12868000,1800.614782,7146.448049
Kanagawa,8943000,2383.038975,3752.771186
Osaka,8801000,1923.151529,4576.342460
Aichi,7418000,5164.400005,1436.372085
Saitama,7130000,3794.036890,1879.264806


다음 코드 셀을 사용하여 인구 밀도를 시각화하는 동적 지도를 생성하세요.

In [ ]:
# Create a base map
m_3 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

# Your code here: create a choropleth map to visualize population density
Choropleth(geo_data=prefectures['geometry'].__geo_interface__,
           data=stats['density'],
           key_on="feature.id",
           fill_color='YlGnBu',
           legend_name='Population density (per square kilometer)'
          ).add_to(m_3)

# View the map
embed_map(m_3, 'q_3.html')

세 개의 지방은 다른 지역보다 상대적으로 더 높은 인구 밀도를 가지고 있나요? 그 지역은 전국적으로 분산되어 있나요, 아니면 거의 동일한 지리적 지역에 위치해 있나요? (*일본 지리에 익숙하지 않은 경우, [이 링크](https://en.wikipedia.org/wiki/Prefectures_of_Japan)를 사용하여 질문에 대답하는 데 도움이 될 수 있습니다.*)

### 4) 고밀도 지방 중에서 강진이 높은 지방은 어디인가요?

방진 보강이 필요한 지방을 제안하기 위해 맵을 생성하세요. 맵은 인구 밀도와 지진 규모를 시각화해야 합니다.  
번역된 내용에 맞게 이해하셨다면, 다음 코드 셀을 실행하여 고밀도와 지진 규모를 시각화하는 맵을 생성하세요.

In [ ]:
# Create a base map
m_4 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

# Your code here: create a map
# Create a map
def color_producer(magnitude):
    if magnitude > 6.5:
        return 'red'
    else:
        return 'green'

Choropleth(
    geo_data=prefectures['geometry'].__geo_interface__,
    data=stats['density'],
    key_on="feature.id",
    fill_color='BuPu',
    legend_name='Population density (per square kilometer)').add_to(m_4)

for i in range(0,len(earthquakes)):
    folium.Circle(
        location=[earthquakes.iloc[i]['Latitude'], earthquakes.iloc[i]['Longitude']],
        popup=("{} ({})").format(
            earthquakes.iloc[i]['Magnitude'],
            earthquakes.iloc[i]['DateTime'].year),
        radius=earthquakes.iloc[i]['Magnitude']**5.5,
        color=color_producer(earthquakes.iloc[i]['Magnitude'])).add_to(m_4)
        
# Uncomment to see a hint
#q_4.a.hint()

# View the map
embed_map(m_4, 'q_4.html')

추가 지진 보강을 위해 어떤 도를 추천하시겠습니까?

인구 밀도와 지진 규모를 시각화한 지도를 기반으로, 저는 추가 지진 보강을 위해 도쿄도를 추천합니다. 도쿄는 인구 밀도가 높고 고규모 지진이 발생하기 쉬운 지역입니다. 따라서 도쿄에서 지진 보강 조치에 우선순위를 두어 인구의 안전과 회복력을 보장하는 것이 매우 중요합니다.